In [1]:
import gym
import ray
from ray import tune
from ray.rllib.agents.ppo import PPOTrainer
from scouting_gym.scouting_env_real import ScoutingEnvInference
from gym.envs.registration import register

register(
    id='Scouting-inference-v0',
    entry_point='scouting_gym.scouting_env_real:ScoutingEnvInference'
)

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
env = gym.make('Scouting-inference-v0')

print(env.observation_space)

Box(0.0, 8.0, (12,), float32)


In [3]:
env.reset()

array([1.14782512e+00, 1.87971377e+00, 2.46162510e+00, 2.27365375e+00,
       2.22166181e+00, 1.87171507e+00, 1.73173630e+00, 1.43978071e+00,
       1.07583618e+00, 1.10783124e+00, 2.00113762e+00, 2.52112088e-04])

In [10]:
env.reset()
for _ in range(20):
    #obs, reward, done, _ = env.step(1)
    obs, reward, done, _ = env.step([0.1, ])
print(obs)
print(obs.min())
print(obs.max())
env.reset()

[1.66774607 1.84371924 1.92370713 1.65974724 1.63175154 1.89371169
 1.27580571 1.39778721 1.42178345 1.23981118 0.66730959 1.39891089]
0.6673095945213912
1.923707127571106


array([1.74173486, 1.85571754, 1.91370857, 1.6217531 , 1.61575401,
       1.84171975, 1.25580871, 1.38178957, 1.42178345, 1.24980962,
       0.64580395, 1.40554403])

In [5]:
print(obs)

[0.67589712 0.60790741 0.51392174 0.53791809 1.07783592 1.65574789
 1.89971077 1.32179868 0.89186424 0.85786939 0.9990963  0.9972344 ]


In [4]:
ray.init()

2021-01-27 11:08:45,370	INFO services.py:1171 -- View the Ray dashboard at http://127.0.0.1:8265


{'node_ip_address': '192.168.178.60',
 'raylet_ip_address': '192.168.178.60',
 'redis_address': '192.168.178.60:6379',
 'object_store_address': '/tmp/ray/session_2021-01-27_11-08-44_861025_3127294/sockets/plasma_store',
 'raylet_socket_name': '/tmp/ray/session_2021-01-27_11-08-44_861025_3127294/sockets/raylet',
 'webui_url': '127.0.0.1:8265',
 'session_dir': '/tmp/ray/session_2021-01-27_11-08-44_861025_3127294',
 'metrics_export_port': 46608,
 'node_id': '766da95910b34a08d865837f6af95a4678f7f99c'}

In [5]:
config = {
    "env": ScoutingEnvInference,  # or "corridor" if registered above
    "env_config": {
        "corridor_length": 5,
    },
    # Use GPUs iff `RLLIB_NUM_GPUS` env var set to > 0.

    "num_gpus": 1,
    "num_workers": 1,  # parallelism
    "model": {
        "fcnet_hiddens": [128, 128, ],
        #"fcnet_hiddens": tune.grid_search([[64, 64, ], [128, 128, ], [256, 256, ]])
    }
}

In [6]:
def load(checkpoint_path, config):
    """
    Load a trained RLlib agent from the specified path. Call this before testing a trained agent.
    :param path: Path pointing to the agent's saved checkpoint (only used for RLlib agents)
    """
    agent = PPOTrainer(config=config)
    agent.restore(checkpoint_path)
    return agent

def test(agent, env):
    """Test trained agent for a single episode. Return the episode reward"""
    # instantiate env class

    # run until episode ends
    episode_reward = 0
    done = False
    obs = env.reset()
    while not done:
        action = agent.compute_action(obs)
        obs, reward, done, info = env.step(action)
        episode_reward += reward
    env.reset()
    return episode_reward

In [7]:
checkpoint_path = '/home/dschori/ray_results/PPO_2021-01-26_21-34-32/PPO_ScoutingDiscreteTask_e5473_00000_0_2021-01-26_21-34-32/checkpoint_100/checkpoint-100'

In [8]:
agent = load(checkpoint_path=checkpoint_path, config=config)

episode_reward = test(agent=agent, env=env)
episode_reward

2021-01-27 11:08:48,214	INFO trainer.py:591 -- Tip: set framework=tfe or the --eager flag to enable TensorFlow eager execution
2021-01-27 11:08:48,214	INFO trainer.py:616 -- Current log_level is WARN. For more information, set 'log_level': 'INFO' / 'DEBUG' or use the -v and -vv flags.
(pid=3127411) WARNING:tensorflow:From /home/dschori/.local/lib/python3.8/site-packages/tensorflow/python/compat/v2_compat.py:96: disable_resource_variables (from tensorflow.python.ops.variable_scope) is deprecated and will be removed in a future version.
(pid=3127411) Instructions for updating:
(pid=3127411) non-resource variables are not supported in the long term
[WARN] [1611742132.281553]: Inbound TCP/IP connection failed: connection from sender terminated before handshake header received. 0 bytes were received. Please check sender for additional details.
2021-01-27 11:08:58,598	INFO trainable.py:99 -- Trainable.setup took 10.385 seconds. If your trainable is slow to initialize, consider setting reuse_

Traceback (most recent call last):
  File "/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py", line 3418, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-8-bf5c9a0ae309>", line 3, in <module>
    episode_reward = test(agent=agent, env=env)
  File "<ipython-input-6-a38baf8bc7c2>", line 20, in test
    obs, reward, done, info = env.step(action)
  File "/home/dschori/MasterThesis/ackerbot_sim_ws/src/rl-scouting/scouting_gym/src/scouting_gym/scouting_env_real.py", line 133, in step
    self.rate.sleep()
  File "/opt/ros/noetic/lib/python3/dist-packages/rospy/timer.py", line 103, in sleep
    sleep(self._remaining(curr_time))
  File "/opt/ros/noetic/lib/python3/dist-packages/rospy/timer.py", line 134, in sleep
    rospy.rostime.wallsleep(duration)
  File "/opt/ros/noetic/lib/python3/dist-packages/rospy/rostime.py", line 277, in wallsleep
    time.sleep(duration)
KeyboardInterrupt

During handling of the above exception, another 

TypeError: object of type 'NoneType' has no len()

In [9]:
env.reset()



array([0.67389745, 0.72189003, 0.70789218, 3.01154137, 2.4916203 ,
       2.34164333, 2.42363095, 2.79557419, 1.24980974, 1.07983553,
       0.72673438, 1.83775304])

(pid=3127411) [WARN] [1611747849.457501]: Inbound TCP/IP connection failed: connection from sender terminated before handshake header received. 0 bytes were received. Please check sender for additional details.
